In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder, QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from pytorch_tabnet.tab_model import TabNetClassifier
import warnings

# 경고 무시 및 시드 설정
warnings.filterwarnings('ignore')
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# MPS 장치 설정 (Apple Silicon 가속)
if torch.backends.mps.is_available():
    device = 'mps'
    print(f"✅ MPS 가속이 활성화되었습니다.")
else:
    device = 'cpu'
    print(f"⚠️ MPS를 찾을 수 없어 CPU로 실행합니다.")

# ==========================================
# 1. 데이터 로드
# ==========================================
# [누수 방지 1, 2]: 학습과 테스트 데이터를 엄격히 분리하여 로드
train_df = pd.read_csv("../Data/train.csv")
test_df = pd.read_csv("../Data/test.csv")

# ==========================================
# 2. 전처리 함수 (파생변수 생성 - 로직만 공유)
# ==========================================
def preprocess_for_tabnet(df):
    df_copy = df.copy()
    
    # 계산형 파생변수 (누수 없음)
    df_copy['배아_생성_효율'] = df_copy['총 생성 배아 수'] / (df_copy['수집된 신선 난자 수'] + 1)
    df_copy['배아_이식_비율'] = df_copy['이식된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    df_copy['배아_저장_비율'] = df_copy['저장된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    
    # 문자열 결합 파생변수 (누수 없음)
    df_copy['나이×Day5'] = df_copy['시술 당시 나이'].astype(str) + '_' + (df_copy['배아 이식 경과일'] == 5.0).astype(str)
    
    # 불필요 변수 제거
    df_copy = df_copy.drop(['ID', '이식된 배아 수'], axis=1)
        
    return df_copy

# 전처리 적용
train_processed = preprocess_for_tabnet(train_df)
test_processed = preprocess_for_tabnet(test_df)

# 타겟 분리
target_col = '임신 성공 여부'
X_train_full = train_processed.drop(columns=[target_col])
y_train_full = train_processed[target_col].values
X_test = test_processed.copy()

# ==========================================
# 3. 데이터 누수 방지 프로세스 (Imputation, Encoding, Scaling)
# ==========================================

# 컬럼 타입 자동 분류
numeric_cols = X_train_full.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
categorical_cols = X_train_full.select_dtypes(include=['object', 'category']).columns.tolist()

# 3-1. 결측치 처리 (Numeric)
# [누수 방지 6]: Test 데이터 결측치는 Train 데이터의 중앙값으로 채움
imputer_num = SimpleImputer(strategy='median')
X_train_full[numeric_cols] = imputer_num.fit_transform(X_train_full[numeric_cols])
X_test[numeric_cols] = imputer_num.transform(X_test[numeric_cols])

# 3-2. 결측치 처리 (Categorical)
# 범주형 결측치는 'Unknown'이라는 새로운 범주로 채움 (통계값 사용 아님)
X_train_full[categorical_cols] = X_train_full[categorical_cols].fillna("Unknown")
X_test[categorical_cols] = X_test[categorical_cols].fillna("Unknown")

# 3-3. Label Encoding (Test Data Unseen Label 처리 포함)
# [누수 방지 3, 5]: pd.get_dummies 금지, LabelEncoder는 Train에만 fit
categorical_dims = {}
for col in categorical_cols:
    le = LabelEncoder()
    # Train 데이터로만 학습
    le.fit(X_train_full[col].astype(str))
    
    # Train 변환
    X_train_full[col] = le.transform(X_train_full[col].astype(str))
    
    # Test 변환 (Safe Handling)
    # Test에만 있는 새로운 값(Unseen)은 Train의 최빈값(Mode)으로 대체하여 누수 방지 및 에러 예방
    test_values = X_test[col].astype(str).values
    train_mode = le.transform([le.classes_[0]])[0] # Fallback용 (실제로는 최빈값 권장하나 여기선 0번 인덱스 활용)
    
    # Unseen Value Masking
    known_labels = set(le.classes_)
    test_values_safe = [x if x in known_labels else le.classes_[0] for x in test_values]
    
    X_test[col] = le.transform(test_values_safe)
    
    # 카테고리 수 저장 (TabNet 입력용)
    categorical_dims[col] = len(le.classes_)

# 3-4. Scaling (QuantileTransformer - 상관계수 낮추기 핵심)
# [누수 방지 4]: Scaler는 Train 데이터로만 fit
# StandardScaler 대신 QuantileTransformer를 사용하여 분포를 정규분포로 강제 변환
# 이는 트리 모델(AutoGluon)과 딥러닝이 데이터를 보는 방식을 근본적으로 다르게 만듦
scaler = QuantileTransformer(output_distribution='normal', random_state=SEED)
X_train_full[numeric_cols] = scaler.fit_transform(X_train_full[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# 데이터 타입 변환 (메모리 최적화)
X_train_full = X_train_full.astype('float32')
X_test = X_test.astype('float32')

# ==========================================
# 4. TabNet 준비
# ==========================================

# Categorical Feature Index 추출
cat_idxs = [i for i, f in enumerate(X_train_full.columns) if f in categorical_cols]
cat_dims = [categorical_dims[f] for f in categorical_cols]

# 학습/검증 데이터 분리 (Stratified Split)
# [누수 방지 1]: 모델 학습 중 검증을 위해 Train set 내에서 분리 (Test set 건드리지 않음)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full.values, y_train_full, 
    test_size=0.2, random_state=SEED, stratify=y_train_full
)

# 불균형 데이터 가중치 계산
# 계산도 오직 분리된 y_train 만을 사용
num_pos = (y_train == 1).sum()
num_neg = (y_train == 0).sum()
pos_weight = num_neg / num_pos
class_weights = torch.tensor([1.0, pos_weight], dtype=torch.float).to(device)

print(f"Positive Class Weight: {pos_weight:.4f}")

# 2. Label Smoothing 적용을 위한 Loss 함수 교체
class LabelSmoothingLoss(nn.Module):
    def __init__(self, weight=None, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing
        self.weight = weight
    def forward(self, inputs, targets):
        log_probs = torch.nn.functional.log_softmax(inputs, dim=-1)
        targets_smooth = torch.zeros_like(log_probs).scatter_(1, targets.unsqueeze(1), 1 - self.smoothing)
        targets_smooth += self.smoothing / log_probs.size(1)
        loss = (-targets_smooth * log_probs).sum(dim=-1)
        if self.weight is not None:
            # 타겟 클래스에 따라 가중치 적용
            weight_per_sample = self.weight[targets]
            loss = loss * weight_per_sample
        return loss.mean()



# ==========================================
# 5. TabNet 모델 학습 (상관계수 파괴 설정)
# ==========================================

clf = TabNetClassifier(
    n_d=16, n_a=16,             # [변경] 모델 용량을 키워 더 복잡한 패턴 학습 유도
    n_steps=3,                  # [변경] 스텝 수를 줄여 트리 모델과 다른 의사결정 깊이 유도
    gamma=1.5, 
    n_independent=2, 
    n_shared=2,
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=4,              # [변경] 임베딩 차원을 2로 늘려 범주 간 관계 학습 강화
    optimizer_fn=torch.optim.AdamW, # [변경] AdamW 사용 (일반화 성능 향상)
    optimizer_params=dict(lr=2e-2, weight_decay=1e-2),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax',
    device_name=device,         # MPS 설정
    seed=SEED
)

loss_fn = LabelSmoothingLoss(weight=class_weights, smoothing=0.1)

print("TabNet 학습 시작 (Quantile Transform 적용됨)...")
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=100,             
    patience=15,                # 참을성 조금 증가
    batch_size=1024,            
    virtual_batch_size=64,      # [변경] Ghost BN 사이즈를 줄여 노이즈 증가 -> 상관계수 감소 유도
    num_workers=0,
    drop_last=False,
    loss_fn=loss_fn             
)

# ==========================================
# 6. Test Data 예측
# ==========================================
# [누수 방지 2]: Test Data는 학습에 전혀 관여하지 않고 예측에만 사용
preds_proba = clf.predict_proba(X_test.values)
final_probs_tabnet = preds_proba[:, 1]

print(f"\n최종 검증 AUC: {clf.best_cost:.5f}")
print("예측 완료. 'final_probs_tabnet'에 확률값이 저장되었습니다.")

✅ MPS 가속이 활성화되었습니다.
Positive Class Weight: 2.8707
TabNet 학습 시작 (Quantile Transform 적용됨)...
epoch 0  | loss: 0.94636 | train_auc: 0.69729 | valid_auc: 0.69689 |  0:00:42s
epoch 1  | loss: 0.91347 | train_auc: 0.72641 | valid_auc: 0.72295 |  0:01:25s
epoch 2  | loss: 0.90889 | train_auc: 0.73052 | valid_auc: 0.72653 |  0:02:07s
epoch 3  | loss: 0.90661 | train_auc: 0.73264 | valid_auc: 0.72761 |  0:02:49s
epoch 4  | loss: 0.90472 | train_auc: 0.73318 | valid_auc: 0.72921 |  0:03:32s
epoch 5  | loss: 0.90454 | train_auc: 0.73412 | valid_auc: 0.7303  |  0:04:14s
epoch 6  | loss: 0.90373 | train_auc: 0.73433 | valid_auc: 0.72898 |  0:04:56s
epoch 7  | loss: 0.90253 | train_auc: 0.73629 | valid_auc: 0.7306  |  0:05:38s
epoch 8  | loss: 0.90221 | train_auc: 0.7373  | valid_auc: 0.73173 |  0:06:20s
epoch 9  | loss: 0.90094 | train_auc: 0.73496 | valid_auc: 0.72957 |  0:07:02s
epoch 10 | loss: 0.90054 | train_auc: 0.73776 | valid_auc: 0.7316  |  0:07:45s
epoch 11 | loss: 0.90031 | train_auc: 0.

In [2]:
# ==========================================
# 7. 결과 추출
# ==========================================
from datetime import datetime
now = datetime.now().strftime('%m%d_%H%M')
file_name = f"TN_{now}_submission.csv"

submission = pd.read_csv("../Data/sample_submission.csv")
submission['probability'] = final_probs_tabnet
submission.to_csv(file_name, index=False)

print(f"완료! 결과 저장됨: {file_name}")
print(f"Best Validation AUC: {clf.best_cost}")

완료! 결과 저장됨: TN_0211_0120_submission.csv
Best Validation AUC: 0.7349332536397428
